In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Smoothie_King/adjust_for_pending_stores'

In [2]:
excel_pending_stores=pd.ExcelFile("./Pending Stores 2019 - Vendors - 8.20.19.xlsx")
excel_pending_stores.sheet_names

['Pending Stores Master List']

In [3]:
df_pending_stores=excel_pending_stores.parse("Pending Stores Master List",dtype=str,skiprows=1)

In [4]:
today=datetime.datetime.now().date()
today_year=today.year
today_month=today.month


In [5]:
start_index_new_location=df_pending_stores[df_pending_stores.iloc[:,0]=="NEW LOCATIONS"].index[0]
end_index_new_location=df_pending_stores[df_pending_stores.iloc[:,0]=="RELOCATIONS"].index[0]
df_new_stores=df_pending_stores.iloc[start_index_new_location:end_index_new_location,:]
df_new_stores=df_new_stores[df_new_stores['Store #']!="nan"]
df_new_stores=df_new_stores[df_new_stores['Opening Target Date']!="TBD"]
df_new_stores['Opening Target Date']=df_new_stores['Opening Target Date'].apply(lambda x: x.split(" ")[0])
df_new_stores=df_new_stores.reset_index()
del df_new_stores['index']

df_new_stores['Opening Target Date']=df_new_stores['Opening Target Date'].replace("11/31/2019","11/30/2019")

for ind,row in df_new_stores.iterrows():
    try:
        df_new_stores.loc[ind,'Opening Target Date']=datetime.datetime.strptime(row['Opening Target Date'],"%Y-%m-%d").date()
    except:
        df_new_stores.loc[ind,'Opening Target Date']=datetime.datetime.strptime(row['Opening Target Date'],"%m/%d/%Y").date()
df_new_stores=df_new_stores[df_new_stores['Opening Target Date']<=datetime.date(today_year,today_month,15)]

In [6]:
start_index_relocation=df_pending_stores[df_pending_stores.iloc[:,0]=="RELOCATIONS"].index[0]
end_index_relocation=df_pending_stores[df_pending_stores.iloc[:,0]=="TRANSFER OF OWNERSHIP"].index[0]
df_relocation_stores=df_pending_stores.iloc[start_index_relocation:end_index_relocation,:]
df_relocation_stores=df_relocation_stores[df_relocation_stores['Store #']!="nan"]


In [7]:
closing_store_list=['1005']

In [8]:
# Revise the "verified store list every month", the used lat/lng
# These look no need to change
df_relocation_stores

,Opening Target Date,Store #,Cleaner Blending,Store Type,Store Franchisee,Address 1,City,State,Zip Code,Franchisee Personal Cell Phone Number,Business E-mail,Ops Manager,Ops Coach,FMM
56,TBD,1175,nan,ECDT,Tim Mahon,Blending Soon at 15825 FOUNTAIN PLAZA DRIVE,Ellisville,MO,63011,314-825-0996,smoothiekingmo@gmail.com,Michael,nan,Tina
57,TBD,20,nan,DT,David Magri,3366 PARIS ROAD,CHALMETTE,LA,70043,504-288-5863,wafsking@aol.com,Jerome,Jeanne,Sara


In [9]:
writer_folder="./output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [10]:
excel_output=pd.ExcelFile("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190902/SmoothieKing_TA_of_3_miles_zips_JL_2019-09-03.xlsx")
excel_output.sheet_names


['summary_by_TA', 'output_TA_by_store', 'zip_TA', 'summary_by_store_count']

In [11]:
df_by_store=excel_output.parse("output_TA_by_store",dtype=str)
df_by_store.shape

(935, 18)

In [12]:
df_by_store=df_by_store[~df_by_store['storenumber'].isin(closing_store_list)]

In [13]:
new_store_list=df_new_stores['Store #'].unique().tolist()
df_by_store[df_by_store['storenumber'].isin(new_store_list)]

# All of the new stores need to be added

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA


In [14]:
df_by_store.head(2)

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA
0,0000,7215 Veterans Memorial Blvd.,Metairie,LA,70003,1973-07-07 00:00:00,30.00589291,-90.22051644,Franchised,Steve Shields,70.183.114.37,Y,Same,nan,30.00589291,-90.22051644,"['70060', '70006', '70141', '70062', '70033', ...","14_['0000', '0002', '0003', '0006', '0010', '0..."
1,0002,4539 Veterans Blvd.,Metairie,LA,70006,1989-09-11 00:00:00,30.00581824,-90.18467266,Franchised,Steve Shields,108.65.143.116,N,Same,nan,30.00581824,-90.18467266,"['70060', '70011', '70004', '70006', '70009', ...","14_['0000', '0002', '0003', '0006', '0010', '0..."


In [15]:
TA_name=df_by_store[['storenumber','TA']].drop_duplicates()
TA_name_count=TA_name.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
TA_name_list=TA_name.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
TA_name=pd.merge(TA_name_count,TA_name_list,on="TA",how="outer")
TA_name['store_count']=TA_name['store_count'].apply(lambda x: str(x))
TA_name['store_list']=TA_name['store_list'].apply(lambda x: str(x))
TA_name['TA_new_name']=TA_name['store_count']+"_"+TA_name['store_list']
TA_name_dict=TA_name.set_index("TA").to_dict()['TA_new_name']


df_by_store['TA']=df_by_store['TA'].apply(lambda x: TA_name_dict[x])
df_by_store=df_by_store.rename(columns={"TA":"TA_new"})


In [16]:
zip_TA=excel_output.parse("zip_TA",dtype=str)
zip_TA=zip_TA[~zip_TA['store'].isin(closing_store_list)]

In [17]:
sorted(zip_TA['Co-Op/Regional'].unique().tolist())

['Atlanta',
 'Baton Rouge',
 'Biloxi',
 'Charlotte',
 'Cincinnati',
 'Dallas-Fort Worth',
 'Houston',
 'Jackson',
 'Jacksonville',
 'Kansas City',
 'Memphis',
 'Miami-Ft. Lauderdale',
 'Mobile-Pensacola-Destin',
 'Nashville',
 'New Orleans',
 'Orlando',
 'San Antonio',
 'St. Louis',
 'Tampa',
 'Unknown',
 'Washington DC',
 'nan']

In [18]:
zip_TA['TA']=zip_TA['TA'].apply(lambda x: TA_name_dict[x])
zip_TA=zip_TA.rename(columns={"TA":"TA_new"})


In [19]:
dict_new_stores={}

# Blending Soon at 12 ROUTE 9, UNIT 103B	Marlboro	NJ	07751
dict_new_stores.update({"1680":{"GoogleAddress":"12 U.S. 9 #103B Morganville, NJ 07751","Google_LatLng":[40.3625614,-74.3036912]}})

# Blending Soon at 7850 NW 25th St	Doral	FL	33122
dict_new_stores.update({"1238":{"GoogleAddress":"7850 NW 25th St Doral, FL 33122","Google_LatLng":[25.7956288,-80.3244997]}})

# Blending Soon at 10800 Kingston Pike	Knoxville	TN	37934
dict_new_stores.update({"1676":{"GoogleAddress":"10800 Kingston Pike Knoxville, TN 37934","Google_LatLng":[35.8936577,-84.1412263]}})

# Blending Soon at 14550 WESTHEIMER ROAD, SUITE 100	Houston	TX	77077
dict_new_stores.update({"1728":{"GoogleAddress":"14550 Westheimer Rd #100 Houston, TX 77077","Google_LatLng":[29.7360888,-95.6435565]}})

# Blending Soon at 1514 Jefferson Hwy	New Orleans	LA	70121
dict_new_stores.update({"1809":{"GoogleAddress":"1514 Jefferson Hwy Jefferson, LA 70121","Google_LatLng":[29.960791,-90.1448282]}})

# Blending Soon at ALLEN PARK PLAZA	Allen Park	MI	48101
dict_new_stores.update({"1761":{"GoogleAddress":"3391 Fairlane Dr, Allen Park, MI 48101","Google_LatLng":[42.2825901,-83.2049022]}})

# Blending Soon at 9170 N. SILVERBELL RD.	Tucson	AZ	85743
dict_new_stores.update({"1803":{"GoogleAddress":"9170 N Silverbell Rd Tucson, AZ 85743","Google_LatLng":[32.3737145,-111.1369671]}})

# Blending Soon at 1231 WILLIAM D. TATE AVENUE	Grapevine	TX	76051	
dict_new_stores.update({"1687":{"GoogleAddress":"1231 William D Tate Ave Grapevine, TX 76051","Google_LatLng":[32.9285253,-97.0850029]}})

# Blending Soon at 4986 STEINER WAY	Grovetown	GA	30813
dict_new_stores.update({"1805":{"GoogleAddress":"4986 Steiner Way Grovetown, GA 30813","Google_LatLng":[33.4809163,-82.1984349]}})



In [20]:
df_new_stores['GoogleAddress']=df_new_stores['Store #'].apply(lambda x: dict_new_stores[x]['GoogleAddress'])
df_new_stores['GoogleLatitude']=df_new_stores['Store #'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][0])
df_new_stores['GoogleLongitude']=df_new_stores['Store #'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][1])


In [21]:
df_new_stores['zips_3_miles']=np.nan

for ind,row in df_new_stores.iterrows():
    store_center=(row['GoogleLatitude'],row['GoogleLongitude'])
    zip_list=[]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=3:
            zip_list.extend([zip_cd])
    df_new_stores.loc[ind,'zips_3_miles']=str(zip_list)


In [22]:
cols=zip_TA.columns.tolist()
zip_ta_new_stores=pd.DataFrame(columns=zip_TA.columns.tolist())
for ind, row in df_new_stores.iterrows():
    store=row['Store #']
    zip_list=eval(row['zips_3_miles'])
    df=pd.DataFrame({"zip":zip_list},index=[store]*len(zip_list)).reset_index().rename(columns={"index":"store"})
    df['TA_new']="new_stores"
    df['Primary_DMA']="new_stores"
    df['Co-Op/Regional']="Unknown"
    zip_ta_new_stores=zip_ta_new_stores.append(df)
zip_ta_new_stores=zip_ta_new_stores[cols]

In [23]:
zip_TA=zip_TA.append(zip_ta_new_stores)
df_TA_zips=zip_TA.copy()

In [24]:
rfa_mapping=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/RFA_region_20190128.xlsx",dtype=str)
rfa_mapping=rfa_mapping[['Store: Store Number  ↑','DMA','Co-Op/Regional']]
rfa_mapping=rfa_mapping.rename(columns={"Store: Store Number  ↑":"store"})


Austin_stores=rfa_mapping[rfa_mapping['DMA'].apply(lambda x: x.lower()=="austin")]

In [25]:
for region,df_group in df_TA_zips.groupby("Co-Op/Regional"):
    if region!="UnknowRegion":
        df_group=df_group[['zip','Co-Op/Regional']].drop_duplicates()
        df_group.to_csv(writer_folder+"zips_for_the_%s.csv"%region,index=False)
    else:
        df_austin=df_group[df_group['store'].isin(Austin_stores['store'].tolist())]
        df_unknown=df_group[~df_group['store'].isin(Austin_stores['store'].tolist())]

        df_austin.to_csv(writer_folder+"zips_for_the_%s.csv"%"AustinDMA",index=False)
        df_unknown.to_csv(writer_folder+"zips_for_the_%s.csv"%"UnknownRegion",index=False)
df_output_overall_zips=df_TA_zips[['zip']].drop_duplicates()
df_output_overall_zips.to_csv(writer_folder+"overall_zips_in_TA.csv",index=False)

In [26]:
writer=pd.ExcelWriter(writer_folder+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_TA_zips.to_excel(writer,"df_TA_zips",index=False)
df_new_stores.to_excel(writer,"new_stores_add_on",index=False)
df_by_store.to_excel(writer,"df_by_store",index=False)
rfa_mapping=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/RFA_region_20190128.xlsx",dtype=str)
rfa_mapping.to_excel(writer,"RFA_region_20190128",index=False)
writer.save()